In [1]:
import scipy.interpolate as interp
from ruspy.estimation.estimation_transitions import create_transition_matrix
from ruspy.model_code.cost_functions import calc_obs_costs
from ruspy.model_code.cost_functions import lin_cost
from ruspy.model_code.fix_point_alg import calc_fixp
from config import DATA_DIR
from global_vals_funcs import VAL_RESULTS

In [2]:
# Load data. This step will be replaced when we have different data.
df = performance_dataframe()
df_meaned = df.mean(level=0)

In [4]:
# Load the probability grid from the data.
grid_probs = pkl.load(open(f"{DATA_DIR}/grid.pkl", "rb"))
samples = pkl.load(open(f"{DATA_DIR}/samples.pkl", "rb"))

In [41]:
grid_probs

array([[0.005, 0.005, 0.99 ],
       [0.005, 0.1  , 0.895],
       [0.005, 0.2  , 0.795],
       [0.005, 0.3  , 0.695],
       [0.005, 0.4  , 0.595],
       [0.005, 0.5  , 0.495],
       [0.005, 0.6  , 0.395],
       [0.005, 0.7  , 0.295],
       [0.005, 0.8  , 0.195],
       [0.005, 0.9  , 0.095],
       [0.005, 0.99 , 0.005],
       [0.1  , 0.005, 0.895],
       [0.1  , 0.1  , 0.8  ],
       [0.1  , 0.2  , 0.7  ],
       [0.1  , 0.3  , 0.6  ],
       [0.1  , 0.4  , 0.5  ],
       [0.1  , 0.5  , 0.4  ],
       [0.1  , 0.6  , 0.3  ],
       [0.1  , 0.7  , 0.2  ],
       [0.1  , 0.8  , 0.1  ],
       [0.2  , 0.005, 0.795],
       [0.2  , 0.1  , 0.7  ],
       [0.2  , 0.2  , 0.6  ],
       [0.2  , 0.3  , 0.5  ],
       [0.2  , 0.4  , 0.4  ],
       [0.2  , 0.5  , 0.3  ],
       [0.2  , 0.6  , 0.2  ],
       [0.2  , 0.7  , 0.1  ],
       [0.3  , 0.005, 0.695],
       [0.3  , 0.1  , 0.6  ],
       [0.3  , 0.2  , 0.5  ],
       [0.3  , 0.3  , 0.4  ],
       [0.3  , 0.4  , 0.3  ],
       [0.

In [39]:
df_meaned.loc[:, 0.0] - df_meaned.loc[:, 0.1]

id_grid
0      -0.283448
1       0.067361
2       0.640821
3       1.055440
4       1.602597
5       3.210129
6       3.531142
7       3.735152
8       3.720133
9       2.659549
10      0.287734
11      0.979060
12      1.714588
13      2.191954
14      2.774590
15      2.923306
16      4.719870
17      5.076685
18      4.385009
19      3.556092
20      3.139155
21      3.724823
22      3.738771
23      4.283365
24      5.167975
25      7.055926
26      6.568014
27      5.741855
28      5.239532
29      5.542003
30      6.161285
31      6.775567
32      8.232594
33      8.499916
34      7.300975
35      8.334295
36      8.491474
37      8.711452
38     11.662742
39     10.993177
40     10.092224
41     11.483353
42     10.571616
43     11.162493
44     13.091225
45     12.077155
46     -2.224277
47     -6.140130
48      2.768408
49      3.102475
50     -3.876606
51    -10.169596
52      6.255646
53    -15.834267
54    -11.538364
55    -49.420498
56    218.256778
dtype: float64

In [33]:
one_dim_eval_grid =  np.append(np.array([0.005]), np.arange(0.01, 1, 0.01))
one_dim_eval_grid

array([0.005, 0.01 , 0.02 , 0.03 , 0.04 , 0.05 , 0.06 , 0.07 , 0.08 ,
       0.09 , 0.1  , 0.11 , 0.12 , 0.13 , 0.14 , 0.15 , 0.16 , 0.17 ,
       0.18 , 0.19 , 0.2  , 0.21 , 0.22 , 0.23 , 0.24 , 0.25 , 0.26 ,
       0.27 , 0.28 , 0.29 , 0.3  , 0.31 , 0.32 , 0.33 , 0.34 , 0.35 ,
       0.36 , 0.37 , 0.38 , 0.39 , 0.4  , 0.41 , 0.42 , 0.43 , 0.44 ,
       0.45 , 0.46 , 0.47 , 0.48 , 0.49 , 0.5  , 0.51 , 0.52 , 0.53 ,
       0.54 , 0.55 , 0.56 , 0.57 , 0.58 , 0.59 , 0.6  , 0.61 , 0.62 ,
       0.63 , 0.64 , 0.65 , 0.66 , 0.67 , 0.68 , 0.69 , 0.7  , 0.71 ,
       0.72 , 0.73 , 0.74 , 0.75 , 0.76 , 0.77 , 0.78 , 0.79 , 0.8  ,
       0.81 , 0.82 , 0.83 , 0.84 , 0.85 , 0.86 , 0.87 , 0.88 , 0.89 ,
       0.9  , 0.91 , 0.92 , 0.93 , 0.94 , 0.95 , 0.96 , 0.97 , 0.98 ,
       0.99 ])

In [34]:
eval_grid = create_prob_grid(one_dim_eval_grid, 3)
eval_grid

array([[0.005, 0.005, 0.99 ],
       [0.005, 0.01 , 0.985],
       [0.005, 0.02 , 0.975],
       ...,
       [0.98 , 0.005, 0.015],
       [0.98 , 0.01 , 0.01 ],
       [0.99 , 0.005, 0.005]])

# Interpolation and apply criteria

In [35]:
omega_vals = df_meaned.loc[:, 0.1].to_numpy()
omega_vals.shape

(57,)

In [36]:
np.log(0.005)

-5.298317366548036

In [37]:
interp.griddata(grid_probs[:, :2], omega_vals, eval_grid[:, :2], method="linear")

array([-84149.04846702, -84041.36854556, -83826.00870265, ...,
        -7248.17442892,  -6492.40407053,  -5127.57832264])

In [29]:
grid_probs[-1, :]

array([0.99 , 0.005, 0.005])

In [28]:
omega_vals

array([-84149.04846702, -82103.12995931, -79901.8551214 , -77647.02686461,
       -75335.57302008, -72965.96876805, -70529.65671891, -68024.24665472,
       -65444.2856395 , -62784.1377063 , -60314.42622636, -79823.37621476,
       -77674.63124476, -75355.44203741, -72976.04571653, -70530.83583672,
       -68025.22377555, -65433.21214939, -62755.76925518, -59988.99053201,
       -75018.45059884, -72741.03455963, -70279.44745778, -67745.686563  ,
       -65130.87268396, -62440.88036552, -59646.08892115, -56750.0089129 ,
       -69906.66900633, -67474.49396847, -64838.34295022, -62112.23967109,
       -59303.03313383, -56371.461096  , -53320.39229255, -64419.79070332,
       -61798.48238563, -58942.30557997, -55989.36011693, -52895.54483529,
       -49660.86213521, -58445.82411476, -55581.97125437, -52438.10374124,
       -49168.69777255, -45708.93640301, -51859.81559756, -48682.77308216,
       -45126.45719213, -41386.52457161, -44385.93798096, -40734.16022616,
       -36581.00479878, -

In [16]:
omega_interpol_vals = np.zeros((grid_probs.shape[0], len(omega_grid)))
for id_grid, gridpoint in enumerate(grid_probs):
    omega_interpol_vals[id_grid, :] = interp.griddata(VAL_STRATS, omega_vals[id_grid, :], omega_grid, method="linear")

In [17]:
omega_interpol_vals.shape

(55, 100)

In [18]:
final_interpol_vals = np.zeros((adapted_grid.shape[0], len(omega_grid)))
for omega_id in range(len(omega_grid)):
    final_interpol_vals[:, omega_id] = interp.griddata(grid_probs[:, :2], omega_interpol_vals[:, omega_id], adapted_grid[:, :2], method="linear")

## Minimax regret

In [19]:
# omegas_mean = np.mean(omega_interpol_vals, axis=0)
# normalized_mean = filter_normalize(omegas_mean)


In [20]:
# regrets_omega = df_meaned.loc[:, "best"].to_numpy().reshape(55, 1) - omega_interpol_vals
# max_regret = np.max(regrets_omega, axis=0)
# normalized_regret = filter_normalize(max_regret * -1)

In [21]:
get_decision_rule_df()

,months,disc_strategy,exp_value
0,0,0.000000,-46985.962594
1,1000,-5051.336957,-46985.962594
2,2000,-9653.143746,-46985.962594
3,3000,-13812.102103,-46985.962594
4,4000,-17579.902280,-46985.962594
...,...,...,...
96,96000,-53390.028609,-46985.962594
97,97000,-53390.372477,-46985.962594
98,98000,-53390.683862,-46985.962594
99,99000,-53390.965687,-46985.962594
